### 시계열 데이터란?

시계열 데이터란 시간의 흐름에 따라 순차적으로 수집된 데이터를 의미한다. 보통 일정한 시간 간격으로 관측된 값들이며, 각 데이터 포인트는 해당 시점을 기준으로 한다.    

### RNN 구조와 동작

RNN은 입력 시퀀스를 시간 순서대로 하나씩 처리하며 과거의 정보를 담은 hidden state를 계속 갱신해 나가는 구조이다. 셀 구조를 반복해서 사용하기 때문에 동일한 가중치들을 매 time step에서 공유한다. 과거의 정보를 매 time step에서 적용하기 때문에 각 시점은 과거 정보를 함께 사용해 값을 출력한다.     

RNN 셀 내부 동작    
입력 벡터 x와 이전 hidden state를 받아 새로운 hidden state 값을 출력하게 된다. 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 1. 하이퍼파라미터 설정
input_size = 10      # 하나의 시점t에서의 특성 수, 즉 입력 벡터의 차원
hidden_size = 20     # hidden state 크기, 예) 20차원 벡터로 지정
output_size = 5      # 출력 크기 (예: 분류 클래스 수)
seq_len = 7          # 시퀀스 길이, 예) 7개 시점의 데이터
batch_size = 16      # 배치 크기, 예) 16개 샘플을 한 번에 처리
num_epochs = 10
learning_rate = 0.001

# 2. RNN 모델 정의
# Recurrent 층이 하나인 단순한 RNN 모델을 정의합니다.
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size)  # 초기 hidden state, 0으로 초기화
        out, _ = self.rnn(x, h0)  # out: (batch, seq_len, hidden_size)
        out = out[:, -1, :]       # 마지막 시점의 hidden state만 사용
        out = self.fc(out)        # 마지막 시점의 hidden state를 fully connected layer에 통과      
        return out                # 최종 출력

model = SimpleRNN(input_size, hidden_size, output_size)

# 3. 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss() # 다중 클래스 분류를 위한 손실 함수 크로스 엔트로피
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam 옵티마이저

In [ ]:
# 4. 더미 데이터 생성 (예시용)
x = torch.randn(batch_size, seq_len, input_size)           # 입력 데이터
y = torch.randint(0, output_size, (batch_size,))           # 정답 레이블

# 5. 학습 루프
for epoch in range(num_epochs):
    output = model(x)
    loss = criterion(output, y) # output: 모델이 예측한 결과, y: 정답 레이블

    optimizer.zero_grad()       # 기울기 0으로 초기화
    loss.backward()             # 역전파 계산
    optimizer.step()            # 가중치 업데이트    

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.6218
Epoch [2/10], Loss: 1.6108
Epoch [3/10], Loss: 1.6000
Epoch [4/10], Loss: 1.5893
Epoch [5/10], Loss: 1.5786
Epoch [6/10], Loss: 1.5681
Epoch [7/10], Loss: 1.5577
Epoch [8/10], Loss: 1.5473
Epoch [9/10], Loss: 1.5371
Epoch [10/10], Loss: 1.5269


: 